In [2]:
import numpy as np
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup, NavigableString
import requests
from numpy import random
import re
from time import sleep
import pickle
from fake_useragent import UserAgent
import os
import pickle

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install())

In [ ]:
driver.get("https://healthunlocked.com/asthmalunguk-asthma/posts") 

# driver.get("https://healthunlocked.com/asthmalunguk-lung/posts")

### Scrollare all'infinito

In [ ]:
more_post = driver.find_element(By.CSS_SELECTOR, "#__next > div > main > div.results.community-page-container > div.results__content > div.community-posts__content > div.results-posts > div.results__see-more-cta > button")

In [ ]:
for i in range(100):
    more_post.click()
    sleep(1)

### Estrazione links

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, "lxml")

In [ ]:
posts = soup.find_all("div", class_="results-post")

In [ ]:
links = list()

In [ ]:
for post in posts:
    if post.find("a") is not None:
        link = post.a["href"]
        links.append(link)

In [ ]:
len(links)

3180

In [ ]:
links[:10]

['/asthmalunguk-asthma/posts/149716068/flixotide-causing-anxiety',
 '/asthmalunguk-asthma/posts/149712616/smoking',
 '/asthmalunguk-asthma/posts/149711216/asthma-appointment',
 '/asthmalunguk-asthma/posts/149707264/in-hospital-again-over-my-oxygen-levels-dropping',
 '/asthmalunguk-asthma/posts/149705176/fostair-joint-pai9n',
 '/asthmalunguk-asthma/posts/149701361/side-effects-report-them-and-yellow-card-scheme-uk',
 '/asthmalunguk-asthma/posts/149699446/asthma-flare-up-and-hoarse-sore-throat-post-covid',
 '/asthmalunguk-asthma/posts/149699042/advice-on-relvar-ellipta-92-22',
 '/asthmalunguk-asthma/posts/149698276/chest-clinic',
 '/asthmalunguk-asthma/posts/149697669/action-plans']

In [ ]:
# with open('links_asthmaUK.pickle', 'wb') as handle:
#     pickle.dump(links, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('links_asthmaUK_2.pickle', 'rb') as handle:
    links = pickle.load(handle)

### Functions to extract data

In [3]:
with open('links_asthmaUK_2.pickle', 'rb') as handle:
    links_asthma = pickle.load(handle)

In [32]:
soup

<!DOCTYPE html>
<html lang="en"><head><meta charset="utf-8"/><meta content="width=device-width,initial-scale=1" name="viewport"/><title>How can asthma breathlessness be diag... - Asthma Community ...</title><meta content="How can asthma breathlessness be diag... - Asthma Community ..." property="og:title"/><meta content="I need help as my daughter suffers with severe breathlessness and also had breathlessness attacks which are asthma attacks. I need advice from anyone how to prove this to the medical team that this is..." name="description"/><meta content="I need help as my daughter suffers with severe breathlessness and also had breathlessness attacks which are asthma attacks. I need advice from anyone how to prove this to the medical team that this is..." property="og:description"/><meta content="https://images.hu-production.be/group-logos/db2929b4-ad19-4cb1-8134-835a4ffc3ba1_140x140@2x.jpg" property="og:image"/><meta content="https://images.hu-production.be/group-logos/db2929b4-ad19

In [4]:
def get_post_data(soup, class_, post_data):
    post = soup.find("div", class_ = class_)
    if post.find("div", class_="sc-6458ca6f-1 eVwnBf").text == "Hidden•":
        post_data["author"].append("Hidden")
    elif post.find("a", class_ = "author") is None:
        post_data["author"].append(post.find_all("a")[1].text)
    else: 
        author = post.find("a", class_ = "author").text
        post_data["author"].append(author.replace("•",""))
    post_data["url"].append(url)
    post_data["title"].append(post.find("h1").text)
    post_data["text"].append(post.find("div", class_ = "post-body").text)
    post_data["time"].append(post.find("time")["title"])
    post_id = f"p_{counter_post}"
    post_data["id"].append(post_id)
    post_like = post.find("button", class_ = "sc-27703fc0-0 jmBqGz").text
    post_data["likes"].append(post_like)
    if type(post.find("div", class_ = "sc-6458ca6f-1 eVwnBf")) == type(None):
        post_data["replies"].append("0 Replies")
    else:
        post_data["replies"].append(post.find("div", class_ = "sc-6458ca6f-1 eVwnBf").text)
    return post, post_data

In [31]:
soup.find("div", class_ = "post-container")

<div class="post-container"><header class="sc-3db2bd78-0 kvqRDh post-header"><h1 class="sc-3db2bd78-1 iDnez" id="post-heading">How can asthma breathlessness be diagnosed? Is there any way to get doctors to diagnose breathlessness to prove its asthma related? </h1><a aria-label="View My_fairy profile" class="userlink-avatar-link" href="/user/My_fairy"><img alt="My_fairy profile image" class="lazy sc-70d59da5-0 dffKiy" data-src="https://images.hu-production.be/avatars/5bc3b69f-3475-4a65-88a4-1e22a6b9ddbf_small@2x_100x100.jpg" height="40" width="40"/></a><div class="sc-11e31e81-0 cogFXz"><div class="sc-11e31e81-1 csxiUp"><a aria-label="View My_fairy profile" class="author" href="/user/My_fairy">My_fairy</a><span class="sc-11e31e81-2 fPdbND hidden-sm">•</span></div><div class="sc-11e31e81-1 csxiUp"><time datetime="2023-07-21T09:57:37.000Z" title="Friday, July 21st, 2023 9:57 AM">5 months ago</time><span class="sc-11e31e81-2 fPdbND">•</span><a href="/asthmalunguk-asthma/posts/149772654/how-

In [5]:
def get_comment_data(comment_, comments_data):
    if comment_.find("div", class_ = "sc-67742806-1 jlVtsa") is not None:
        if comment_.find("div", class_ = "sc-67742806-1 jlVtsa").find("button") is not None:
            if comment_.find("div", class_ = "sc-67742806-1 jlVtsa").find("button").text == "Hidden":
                comments_data["author"].append("Hidden")
        else:
            comments_data["author"].append(comment_.find_all("a")[1].text)
    else:
        comments_data["author"].append(comment_.find_all("a")[1].text)
    comments_data["time"].append(comment_.find("time")["title"])
    children = [child for child in comment_.children]
#     comments_data["text"].append(get_text_from_children(children[1:]))
    texts = comment_.find_all("p")
    text = "".join([str(element.text) for element in comment_.find_all("p")])
    comments_data["text"].append(text)
    comments_data["id"].append(f"tc_{counter_top_comment}")           #tc = top-comment
    comments_data["parent_id"].append(post_data["id"][-1])
    comment_like = comment_.find("button", class_ = "sc-27703fc0-0 jmBqGz").text
    comments_data["likes"].append(comment_like)
    comments_data["url"].append(url)
    return comments_data

In [6]:
def get_sub_comment_data(sub_comment, comments_data, change_block):
    children = [child for child in sub_comment.children]
    authors = [string for string in children[0].strings]
    if "Administrator" in authors:
        authors.remove("Administrator")
    if "Partner" in authors:
        authors.remove("Partner")
    if "Verified User" in authors:
        authors.remove("Verified User")
    if "Community Ambassador" in authors:
        authors.remove("Community Ambassador")
    if "British Lung Foundation" in authors:
        authors.remove("British Lung Foundation")
    if "Volunteer" in authors:
        authors.remove("Volunteer")
    comment_author = authors[0]
    commented_author = authors[4]
    comments_data["author"].append(comment_author)
#     comments_data["text"].append(get_text_from_children(children[1:]))
    texts = sub_comment.find_all("p")
    text = "".join([str(element.text) for element in sub_comment.find_all("p")])
    comments_data["text"].append(text)
    comments_data["time"].append(sub_comment.find("time")["title"])
    comments_data["id"].append(f"s_{counter_comment}")
#     print(sub_comment)
    comments_data["likes"].append(sub_comment.find("button", class_ = "sc-27703fc0-0 jmBqGz").text)
    comments_data["url"].append(url)
#     if comments_data["id"][-2].startswith("f"):
#         index_commented = [loc for loc, val in enumerate(comments_data["author"]) if (val == commented_author)
#                           and (comments_data["id"][-2].startswith("s"))]
#     else:
    if change_block:
        index_commented = [loc for loc, val in enumerate(comments_data["author"]) if val == commented_author]
        for index_ in index_commented:
            if comments_data["id"][index_].startswith("tc"):
                index = index_
    else:
        index_commented = [loc for loc, val in enumerate(comments_data["author"]) if val == commented_author]
        index = index_commented[-1]
        
#     print(index, change_block)
        
    if comments_data["id"][index].startswith("tc") or comments_data["id"][index].startswith("s"):
        comments_data["parent_id"].append(comments_data["id"][index])
    elif comments_data["id"][index].startswith("f"):
        for index in index_commented:
            if comments_data["id"][index].startswith("tc") or comments_data["id"][index].startswith("s"):
                new_index = index
#                 break
        comments_data["parent_id"].append(comments_data["id"][new_index])
    return 

In [7]:
def get_full_indent_data(full_indent, comments_data):
    children = [child for child in full_indent.children]
    authors = [string for string in children[0].strings]
    if "Administrator" in authors:
        authors.remove("Administrator")
    if "Partner" in authors:
        authors.remove("Partner")
    if "Verified User" in authors:
        authors.remove("Verified User")
    if "Community Ambassador" in authors:
        authors.remove("Community Ambassador")
    if "British Lung Foundation" in authors:
        authors.remove("British Lung Foundation")
    if "Volunteer" in authors:
        authors.remove("Volunteer")
    comment_author = authors[0]
    commented_author = authors[4]
#     print(commented_author, comment_author)
    comments_data["author"].append(comment_author)
#     comments_data["text"].append(get_text_from_children(children[1:]))
    texts = full_indent.find_all("p")
    text = "".join([str(element.text) for element in full_indent.find_all("p")])
    comments_data["text"].append(text)
    comments_data["time"].append(full_indent.find("time")["title"])
    comments_data["id"].append(f"f_{counter_fi}")
    comments_data["likes"].append(full_indent.find("button", class_ = "sc-27703fc0-0 jmBqGz").text)
    comments_data["url"].append(url)
    index_commented = [loc for loc, val in enumerate(comments_data["author"]) if val == commented_author]
    if index_commented == []:
        comments_data["parent_id"].append(None)
        missing_authors.append(url)
        flag_none = True
    else:
        index_commented = index_commented[-1]
        comments_data["parent_id"].append(comments_data["id"][index_commented])
        flag_none = False
#     if flag_none:
#         print(flag_none)
#         print(comments_data["text"][-1])
    return flag_none

In [8]:
def reset_dict_create_df(post_data, comments_data, df_post, df_com):
    com_new = pd.DataFrame(comments_data)
    post_new = pd.DataFrame(post_data)
    df_com = df_com.append(com_new, ignore_index = True)
    df_post = df_post.append(post_new, ignore_index = True)
    comments_data = {"text": list(), "author": list(), "likes": list(),
                 "time": list(), "id": list(),
                 "parent_id": list(), "url": list()}
    post_data = {"title": list(), "text": list(), "author": list(),
        "likes": list(), "time": list(), "id": list(), "url": list(), 
             "replies": list()}
    return post_data, comments_data, df_post, df_com

In [9]:
def find_top_comment(children, counter):
    i = False
    counter = 1
    while i is not True:
        counter += 1
        if counter >= len(children):
            return None
        if "class" in children[counter].attrs:
            if "gcfwZj" in children[counter].attrs["class"]:
                i = True
    return counter

In [10]:
def get_limit_comments(descend):#, block): #(top_comment):

    num = []
    class_num = {}
    block_num = {}
#     second_level_com_position = [m.start() for m in re.finditer('<div class="sc-d94c948d-0 ffVYIA"',  "".join([str(i) for i in descend]))]
    second_level_com_position = [m.start() for m in re.finditer('<div class="sc-ece900ce-0 gcfwZj"',  str(descend))]
    
    if second_level_com_position == []:
        return None, None
    
    num += second_level_com_position
    for i in second_level_com_position: 
        class_num[i] = "sc-ece900ce-0 gcfwZj"
        
        #     full_indent_position = [m.start() for m in re.finditer('<div class="sc-d94c948d-0 kbsIKK"',  "".join([str(i) for i in descend]))]
    full_indent_position = [m.start() for m in re.finditer('<div class="sc-ece900ce-0 bifjEl"',  str(descend))]
    if full_indent_position == []:
        return num, class_num
    
    num += full_indent_position
    for i in full_indent_position:
        class_num[i] = "sc-ece900ce-0 bifjEl"

    
    return num, class_num#, block_num

In [11]:
def pieces_to_tag(pieces, start, stop):
#     if stop == None:
#         comment = "".join(str(element) for element in pieces)[start:]
#     else:
#         comment = "".join(str(element) for element in pieces)[start:stop]
    comment = "".join(str(element) for element in pieces)[start:stop]
    soup = BeautifulSoup(comment, "lxml")
    return soup

In [12]:
def get_comment_blocks(descendants, num):
    comments_block = []
    
#     if num == None:
#         comments_block.append(pieces_to_tag(descendants, 0, None))
#         return comments_block
#     first = True
    
    i = 0
    num = sorted(num)
    for i in range(len(num)): 
#         if first == True:
#             comments_block.append(pieces_to_tag(descendants, 0, num[i]))
#             first = False
#         else:
            if i == len(num)-1:
                break
            else:
                comments_block.append(pieces_to_tag(descendants, num[i], num[i+1]))
    comments_block.append(pieces_to_tag(descendants, num[i], None))
    return comments_block

### Data

In [24]:
post_data = {"title": list(), "text": list(), "author": list(),
        "likes": list(), "time": list(), "id": list(), "url": list(), 
             "replies": list()}

In [25]:
comments_data = {"text": list(), "author": list(), "likes": list(),
                 "time": list(), "id": list(),
                 "parent_id": list(), "url": list()}

#### Scraper

In [ ]:
counter_post = 0
counter_top_comment = 0
counter_comment = 0
counter_fi = 0
df_post = pd.DataFrame()
df_com = pd.DataFrame()

cookies = {'huSessID': ''} #INSERT HERE THE personal huSessID

for link in links_asthma:
    
    if counter_comment % 25 == 0:
        post_data, comments_data, df_post, df_com = reset_dict_create_df(post_data, comments_data, df_post, df_com)
    
    url = f"https://healthunlocked.com{link}"
#     req = requests.get(url, cookies = cookies)
    req = requests.get(url, cookies = cookies)
    n_req = 0
    if req.status_code == 410:
        continue
    while (req.status_code != 200) and (n_req < 6):
        sleep(2)
        print(f"richiesta n{n_req} fallita. Status code: {req.status_code}", end = "\r")
        n_req +=1 
        req = requests.get(url, cookies = cookies)
    html = req.text
    soup = BeautifulSoup(html, "lxml")
    
    if type(soup.find("div", class_ = "error-page")) != type(None):
        if "no longer exist" in soup.find("div", class_ = "error-page").text:
            continue
        
    post, post_data = get_post_data(soup, "post-container", post_data)


    counter_post += 1 

    comments = soup.find_all("div", class_ = "sc-ece900ce-0 kYRKGz")   #post-response-item top-level-comment

    for comment in comments:
        children = [i for i in comment.children]
        stop_top_comment = find_top_comment(children, 2)
        if stop_top_comment == None:
            top_comment= pieces_to_tag(children, 0, None)
            get_comment_data(top_comment, comments_data)
            counter_top_comment += 1
            continue
        top_comment = children[:stop_top_comment]
        top_comment = pieces_to_tag(top_comment, 0, None)
        get_comment_data(top_comment, comments_data)
        counter_top_comment += 1

        if len(children) <= 3:
            continue

        for child in children[stop_top_comment:]:
            change_block = True
            n_comment = 0
            num, class_num = get_limit_comments(child)    #find when a comment ends and a new one begins
            blocks = get_comment_blocks(child, num)

            for block in blocks:
                if class_num[sorted(num)[n_comment]] == "sc-ece900ce-0 gcfwZj":
                    if class_num[sorted(num)[n_comment]]:
                        get_sub_comment_data(block, comments_data, change_block)
                        change_block = False
                        n_comment += 1
                        counter_comment += 1 
                elif class_num[sorted(num)[n_comment]] == "sc-ece900ce-0 bifjEl":
                    get_full_indent_data(block, comments_data)
                    n_comment += 1
                    counter_fi += 1

    print(f"siamo al post {counter_post}", end = "\r")

In [287]:
df_post = pd.DataFrame(post_data)
df_post.head()

,title,text,author,likes,time,id,url,replies
0,Good morning Everyone . Just thought I would s...,Some lovely words,Bingo88,Like (28),"Monday, July 24th, 2023 6:54 AM",p_0,https://healthunlocked.com/asthmalunguk-lung/p...,Bingo88•
1,hip op,some weeks ago I commented on a thread regardi...,Yeo58,Like (19),"Sunday, July 23rd, 2023 7:28 PM",p_1,https://healthunlocked.com/asthmalunguk-lung/p...,Yeo58•
2,Lessening effect of Trimbow,"Hello to you all, as the headline says I am no...",ralphb,Like (7),"Sunday, July 23rd, 2023 6:53 PM",p_2,https://healthunlocked.com/asthmalunguk-lung/p...,ralphb•
3,screen shot of u tube id for gorillapod for m...,screen shot for mellywelly doesnt work on mes...,Jaybird19,Like (0),"Sunday, July 23rd, 2023 6:35 PM",p_3,https://healthunlocked.com/asthmalunguk-lung/p...,Jaybird19•
4,Raising COPD Awareness.,Raising COPD Awareness.,Joseph260268,Like (8),"Sunday, July 23rd, 2023 11:39 AM",p_4,https://healthunlocked.com/asthmalunguk-lung/p...,Joseph260268•


In [288]:
df_com = pd.DataFrame(comments_data)
df_com.head()

,text,author,likes,time,id,parent_id,url
0,Thank you Brian for the lovely verse. Hope you...,sassy59,(2),"Monday, July 24th, 2023 7:19 AM",tc_0,p_0,https://healthunlocked.com/asthmalunguk-lung/p...
1,Good morning Carole. Yes alls as good as it ge...,Bingo88,(2),"Monday, July 24th, 2023 7:35 AM",s_0,tc_0,https://healthunlocked.com/asthmalunguk-lung/p...
2,Good to hear you’re doing ok Brian. All fine h...,sassy59,(1),"Monday, July 24th, 2023 7:51 AM",s_1,s_0,https://healthunlocked.com/asthmalunguk-lung/p...
3,Hi Brian! Morning inhalers and coffee! Just wh...,leo60,(1),"Monday, July 24th, 2023 8:33 AM",tc_1,p_0,https://healthunlocked.com/asthmalunguk-lung/p...
4,Hope your doing well too. Always good to hear ...,Bingo88,(1),"Monday, July 24th, 2023 9:00 AM",s_2,tc_1,https://healthunlocked.com/asthmalunguk-lung/p...
